# Interacting with GPIO from MicroBlaze

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
import threading
import multiprocessing
import os
import socket
from multiprocessing import Process, Value, Array, Lock

base = BaseOverlay("base.bit")

In [ ]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpioB(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpioB(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpioB(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpioB(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

In [ ]:
#LED_Status = [0, 0, 2, 3] #Default Status
#LED_Status = [1, 1, 2, 1] #test
#start_LED = False #when connection is made, LED() will start
#running = True #when client breaks connection, LED() sill stop running

sleep_time = 1
frequency = 15 #for the PWM Brightness
Duty_Cycle = [0.04736614270344993, 0.217637640824031, 0.5310492251033824, 1.0] #25%, 50%, 75%, and 100% brightness.
#time_on = duty_cycle * (1/frequency) && time_off = (1-duty_cycle) * (1/frequency)
PWM_on = []
for i in range(4):
    PWM_on.append(Duty_Cycle[i]/frequency)
PWM_off = []
for i in range(4):
    PWM_off.append((1-Duty_Cycle[i])/frequency)
PWM_run_time = frequency*sleep_time

In [ ]:
def LED(LED_Status_pynq,_LL1, running_pynq, _LL2, _Lt0, _Lt1, Array_Change):
    global PWM_on, PWM_off, PWM_run_time, sleep_time
    LED_Status = [0, 0, 2, 3]
    #with _LL1:
        #for i in range(4):
            #ED_Status.append(LED_Status_pynq[i])
    while True:
        #print('LED Status: ',LED_Status[:],'\n running: ',running.value)
        threadlock = _Lt0.acquire(False)
        if threadlock == True:
            with _LL2:
                if running_pynq.value == 0:
                    break
            if Array_Change == False:
                if LED_Status[0] == 0: #IF OFF
                    time.sleep(sleep_time)
                else: #IF ON
                    if LED_Status[3] == 3: #full bright (NO PWM)
                        if LED_Status[2] == 0: #If NO PWM & Cycle Color
                            for i in range(1,4): #cycle write from 1 (blue), 2 (red) and 3 (green)
                                write_gpioB(i,1) #turn on  LED
                                time.sleep(sleep_time) #sleep with ON
                                write_gpioB(i,0) #turn off LED
                                if LED_Status[1] == 1: #if supposed to blink, sleep
                                    time.sleep(sleep_time)
                        else: #IF NO PWM & Solid Color
                            write_gpioB(LED_Status[2],1)
                            time.sleep(sleep_time)
                            if LED_Status[1] == 1: #if supposed to blink, sleep
                                    write_gpioB(LED_Status[2],0)
                                    time.sleep(sleep_time)
                    else: #If NOT full brightness (PWM needed)
                        if LED_Status[2] == 0: #PWM & cycle thru colors
                            for i in range(1,4):
                                for j in range(PWM_run_time):
                                    write_gpioB(i,1) #turn on LED
                                    time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                                    write_gpioB(i,0) #turn off LED
                                    time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                                if LED_Status[1] == 1: #If blinking
                                    time.sleep(sleep_time)
                        else:
                            for i in range(PWM_run_time):
                                write_gpioB(LED_Status[2],1) #turn on LED
                                time.sleep(PWM_on[LED_Status[3]]) #sleep with ON
                                write_gpioB(LED_Status[2],0) #turn off LED
                                time.sleep(PWM_off[LED_Status[3]]) #sleep with OFF
                            if LED_Status[1] == 1: #If Blinking
                                time.sleep(sleep_time)
            else: #If received a message signaling a change:
                with _LL1:
                    LED_Status = LED_Status_pynq
                with _Lt1:
                    Array_Change = False
            _Lt0.release() 
            #time.sleep(.0001) #no sleep time...
        else: #Does not have _Lt0
            with _LL1: #check for status change
                if LED_Status != LED_Status_pynq:
                    with _Lt1: #if status change, force _Lt1 to change LED_Status!
                        Array_Change = True
            with _LL2:
                if running.value == 0:
                    break
            time.sleep(.25)

def Wait_LED(start_p, LED_Status_p, running_p, _LP0, _LP1, _LP2):
    global sleep_time
    print('Waiting for LED to start.')
    threads = []
    while True:
        with _LP0: #will keep this for the remainer of the program once start_p==1
            if start_p.value == 1:
                print("Connors now has control of Ramin's LED!")
                fork1 = threading.Lock() #main lock for threads
                fork2 = threading.Lock() #lock for Array_Change (variable within LED())
                Change2Array = False #variable for above lock ^
                for i in range(2):
                    #fork is the shared lock
                    #i is the user (led number too)
                    t = threading.Thread(target=LED, args=(LED_Status_p, _LP1, running_p, _LP2, fork1, fork2, Change2Array))
                    threads.append(t) #creates the new thread number
                    t.start()
                reset_gpioB()
                for t in threads:
                    t.join()
                print('Multithreading has terminated.')
                break
        #print('waiting... ', start_p)
        time.sleep(sleep_time)
        

In [ ]:
def run_pynq_server_R(start_LED_s, LED_Status_s, running_s,_LS0,_LS1,_LS2):
    global 
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('192.168.2.99', 12345))
    sock.listen()
    conn, addr = sock.accept()
    print("Ramin's Server is waiting for a connection!")
    with conn:
        print("Connected to Ramin's Remote Server!")
        with _LS0:
            start_LED_s.value = 1 #Now we can allow the LED() function to begin. Then drop lock, never get it back.
        while True:
            time.sleep(0.0001)
            data = conn.recv(1024)
            data = data.decode()
            if (data == 'disconnect'):
                print("Client is disconnecting from Ramin's Server.")
                with _LS2:
                    running_s.value = 0
                break
            data = list(data) #Here we convert from string into list, then update all indexes from str to int
            with _LS1:
                for i in range(4): #length of data list ALWAYS = 4, could do 'len(data)' instead though.
                    LED_Status_s[i] = int(data[i])
    print("Client has disconnected from Ramin's Server.")


In [ ]:
start_i = Value('i', 0) #SHARED!
running_i = Value('i',1) #SHARED!
LED_Status_i = Array('i', [1, 0, 2, 2]) #SHARED
_L0 = Lock() #Lock0 for Start
_L1 = Lock() #Lock1 For LED status
_L2 = Lock() #Lock2 for running

p_server = multiprocessing.Process(target=run_pynq_server_R,args=(start_i,LED_Status_i,running_i,_L0,_L1,_L2))
p_server.start()
os.system("taskset -p -c {} {}".format(0, p_server.pid))

p_GPIOB = multiprocessing.Process(target=Wait_LED,args=(start_i,LED_Status_i,running_i,_L0,_L1,_L2))
p_GPIOB.start()
os.system("taskset -p -c {} {}".format(1, p_GPIOB.pid))

p_server.join();
print('Process Server with name, {}, is finished'.format(p_server.name))
p_GPIOB.join();
print('Process GPIO with name, {}, is finished'.format(p_GPIOB.name))

In [ ]:
def test(start_LED_s, LED_Status_s, running_s,_LS0,_LS1,_LS2):
    itera = 0
    while True:
        itera +=1
        if itera>3:
            with _LS0:
                start_LED_s.value = 1 #Now we can allow the LED() function to begin.
            print('start_LED = ',start_LED_s.value)
            print('Led status: ', LED_Status_s[:])
            for i in range(2):
                time.sleep(1)
            print('on:')
            data = ['1','1', '3', '0']
            with _LS1:
                for i in range(4): #length of data list ALWAYS = 4, could do 'len(data)' instead though.
                    LED_Status_s[i] = int(data[i])
            for i in range(5):
                time.sleep(1)
            print('LED status change')
            data = ['1','0', '0', '3']
            with _LS1:
                for i in range(4): #length of data list ALWAYS = 4, could do 'len(data)' instead though.
                    LED_Status_s[i] = int(data[i])
            for i in range(10):
                time.sleep(1)
            print('LED status change')
            data = ['1','1', '1', '2']
            with _LS1:
                for i in range(4): #length of data list ALWAYS = 4, could do 'len(data)' instead though.
                    LED_Status_s[i] = int(data[i])
            for i in range(6):
                time.sleep(1)
            print('About to disconnect')
            for i in range(2):
                time.sleep(1)
            with _LS2:#disconnect
                running_s.value = 0
            print('Disconnected!')
            break
        print(itera)
        time.sleep(1)

In [ ]:
#FOR LOCAL TESTING ONLY!
reset_gpioB()

start_i = Value('i', 0) #SHARED!
running_i = Value('i',1) #SHARED!
LED_Status_i = Array('i', [0, 0, 2, 2]) #SHARED
_L0 = Lock() #Lock0 for Start
_L1 = Lock() #Lock1 For LED status
_L2 = Lock() #Lock2 for running

p_GPIOB = multiprocessing.Process(target=test,args=(start_i,LED_Status_i,running_i,_L0,_L1,_L2))
p_GPIOB.start()
os.system("taskset -p -c {} {}".format(1, p_GPIOB.pid))
p_server = multiprocessing.Process(target=Wait_LED,args=(start_i,LED_Status_i,running_i,_L0,_L1,_L2))
p_server.start()
os.system("taskset -p -c {} {}".format(0, p_server.pid))

p_GPIOB.join();
print('Process Client with name, {}, is finished'.format(p_GPIOB.name))
p_server.join();
print('Process GPIO with name, {}, is finished'.format(p_server.name))